In [12]:
import requests
import re

import gensim
import gensim.downloader as api
# Using Google news pretrained word to vectorized.
#path = api.load("word2vec-google-news-300", return_path=True)

#Download GoogleNews-vectors-negative300.bin from google into the code path
google = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin",binary=True)
vocab = google.vocab.keys()

#Beautiful Soup for scraping content from the website
from bs4 import BeautifulSoup

# for removing stopwords
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nitheesh.annadanam\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nitheesh.annadanam\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nitheesh.annadanam\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
#Function to scrape contetn from google and clean it up.
def cleanup(link):
    
    # get the link
    page = requests.get(link)
    
    #Get the content using beautiful soup
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Get all the content in p tag alone (the content of the article)
      # (This works well in removing ads and other links and more than 95 % of text after this is related to the article )
    
    whitelist = ['p']
    text_elements = [t for t in soup.find_all(text=True) if t.parent.name in whitelist]
    te=" ".join(text_elements)
    
    #Cleaning the text by removing special characters and mutliple spaces and full stops.
    tene=re.sub("[^\\.{1},a-zA-Z {1}]"," ",te)
    tene=tene.replace("/t","")
    tene=re.sub(" +"," ",tene)
    tene=re.sub("\.+", ".",tene)
    
    #removing stop words
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(tene) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    fin =" ".join(filtered_sentence)

    return(fin)




In [14]:
import numpy as np
from scipy import spatial

index2word_set = set(google.wv.index2word)

# Calculating the average of the google vectors for the article.
def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

C:\Users\nitheesh.annadanam\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [18]:
# Link 1 and calculating the average of the vectors
#link1 = cleanup("https://www.ndtv.com/india-news/coronavirus-38-310-fresh-covid-19-cases-in-india-15-lower-than-yesterday-82-67-lakh-total-cases-so-far-1-23-097-deaths-2319732?pfrom=home-coronavirus_outbreak_coverage")
link1 = cleanup("https://www.washingtonpost.com/health/2021/05/30/barbershop-coronavirus-vaccines/?itid=hp-top-table-main")
av1=avg_feature_vector(link1,google,300,index2word_set)


In [19]:
# Link 2 and calculating the average of the vectors
#link2 = cleanup("https://www.indiatoday.in/coronavirus-outbreak/story/coronavirus-another-peak-possible-but-all-depends-behaviour-health-ministry-1737666-2020-11-03")
link2 = cleanup("https://www.theguardian.com/world/2021/jun/01/uk-records-no-covid-19-deaths-in-a-day-raising-hope-for-unlocking")

av2=avg_feature_vector(link2,google,300,index2word_set)

In [21]:
#Calculaying cosine similarity score
# Calculating cosine similarity of the average vectors is being used as a measure of the similarity of topics because
# the google vectors provide an "understanding" of the words being used as similar words are placed closer to each other.
sim = 1 - spatial.distance.cosine(av1, av2)
print("The articles are",round(sim*100,2)," % similar")

The articles are 85.72  % similar


In [ ]:
#The links to try
original link = https://www.ndtv.com/india-news/details-of-dissent-letter-to-sonia-gandhi-steady-decline-no-honest-inspection-2286399
l1= https://www.indiatoday.in/sports/ipl-2020/story/virat-kohli-rcb-dressing-room-speech-video-proud-of-ipl-campaign-fans-expectations-1738811-2020-11-07
l2= https://sports.ndtv.com/ipl-2020/srh-vs-rcb-indian-premier-league-virat-kohli-shares-emotional-post-after-rcbs-ipl-2020-dream-ends-without-silverware-2321920